In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as T
from PIL import Image
import os
import json
from pycocotools.coco import COCO
from transformers import DetrForObjectDetection, DetrImageProcessor
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
from tqdm import tqdm

# --- 1.1. 설정 (Configuration) ---
class Config:
    def __init__(self):
        # 훈련 데이터셋 경로
        self.train_image_dir = '../data/detr/train_images'
        self.train_annotation_file = '../data/detr/annotations/train.json'
        # 테스트 데이터셋 경로
        self.test_image_dir = '../data/detr/test_images'

        # 사용할 DETR 모델 이름
        self.model_name = "facebook/detr-resnet-50" # 사전 훈련된 DETR ResNet-50 모델

        # 결과물 (모델 가중치, 예측 이미지) 저장 경로
        self.output_dir = '../models'

        # 학습 관련 파라미터
        self.epochs = 1  # 실제 학습 시에는 더 많은 Epoch가 필요할 수 있습니다.
        self.batch_size = 1
        self.learning_rate = 1e-4

        # 사용 디바이스 (GPU 사용 가능 시 'cuda', 아니면 'cpu')
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'

        # 예측 시각화 관련 파라미터
        self.viz_num_images = 5 # 테스트 데이터셋에서 시각화할 이미지 수
        self.detection_threshold = 0.9 # 예측 신뢰도 임계값 (이 값 이상인 예측만 시각화)

# Config 인스턴스 생성
config = Config()
print(f"Running on device: {config.device}")
print(f"Output will be saved to: {config.output_dir}")

# --- 1.2. 모델 이미지 프로세서 로드 ---
# 이 프로세서는 DETR 모델 입력에 맞게 이미지를 전처리합니다.
image_processor = DetrImageProcessor.from_pretrained(config.model_name)
print(f"DETR Image Processor loaded: {config.model_name}")

# 이 코드를 실행하기 전에 `train_image_dir`, `train_annotation_file`, `test_image_dir`의 경로를
# 실제 파일이 있는 곳으로 **반드시 수정**해주세요.

c:\Users\kangdongwoo\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on device: cpu
Output will be saved to: ../models/detr_output
DETR Image Processor loaded: facebook/detr-resnet-50


In [2]:
# --- 2.1. 훈련 데이터셋 클래스 (COCO Detection Dataset) ---
class CocoDetection(Dataset):
    def __init__(self, img_folder, ann_file, image_processor):
        self.img_folder = img_folder
        self.coco = COCO(ann_file)
        self.img_ids = list(self.coco.imgs.keys())
        self.image_processor = image_processor

        self.categories = self.coco.loadCats(self.coco.getCatIds())
        self.id_to_label = {c['id']: i for i, c in enumerate(self.categories)}
        self.label_to_id = {i: c['id'] for i, c in enumerate(self.categories)}
        self.label_to_name = {i: c['name'] for i, c in enumerate(self.categories)}

    def __len__(self):
        return len(self.img_ids)

    def __getitem__(self, idx):
        img_id = self.img_ids[idx]
        img_info = self.coco.loadImgs(img_id)[0] # 이미지 정보 로드
        ann_ids = self.coco.getAnnIds(imgIds=img_id)
        anns = self.coco.loadAnns(ann_ids) # 해당 이미지의 모든 원본 어노테이션 로드
        path = img_info['file_name']

        img = Image.open(os.path.join(self.img_folder, path)).convert('RGB')

        # === 핵심 변경 부분 ===
        # image_processor의 annotations 인자에 원본 COCO 포맷의 어노테이션 딕셔너리를 전달합니다.
        # 이 딕셔너리는 'image_id'와 'annotations' (원본 어노테이션 리스트) 키를 포함해야 합니다.
        coco_target = {
            'image_id': img_id,
            'annotations': anns # PyCOCOTools에서 로드한 원본 어노테이션 리스트 그대로 전달
        }

        # 이미지 전처리 및 어노테이션 변환을 image_processor에 맡깁니다.
        # images 인자에는 단일 이미지 객체를, annotations 인자에는 위에서 정의한 coco_target을 리스트로 감싸서 전달
        # 왜 리스트로 감싸는가? image_processor가 배치 처리(list of images)를 기본으로 하기 때문입니다.
        # 단일 이미지를 전달할 때는 [image_obj] 로, 해당 이미지의 어노테이션을 [coco_target_dict] 로 전달합니다.
        encoded_inputs = self.image_processor(images=[img], annotations=[coco_target], return_tensors="pt")

        # encoded_inputs는 'pixel_values'와 'labels' (boxes, class_labels)를 포함하는 딕셔너리
        # 'pixel_values'는 배치 차원 (1)을 가지므로 제거하고, 'labels'는 [ {'boxes': ..., 'class_labels': ...} ] 형태이므로 첫 번째 요소를 사용
        return {
            'pixel_values': encoded_inputs['pixel_values'].squeeze(0), # 배치 차원 제거
            'labels': encoded_inputs['labels'][0] # 리스트에서 유일한 어노테이션 딕셔너리 추출
        }

# 나머지 코드는 이전 답변과 동일하게 유지됩니다.
# detr_collate_fn_train 및 test_dataloader, train_dataloader 생성 부분은 변경 없습니다.
# --- 2.2. 테스트 데이터셋 클래스 (어노테이션 없음) ---
class TestImageDataset(Dataset):
    def __init__(self, img_folder, image_processor, category_map):
        self.img_folder = img_folder
        self.image_processor = image_processor
        self.image_files = [f for f in os.listdir(img_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        self.image_paths = [os.path.join(img_folder, f) for f in self.image_files]
        self.label_to_name = category_map

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        img = Image.open(img_path).convert('RGB')

        encoded_inputs = self.image_processor(images=img, return_tensors="pt")
        return {
            'pixel_values': encoded_inputs['pixel_values'].squeeze(0),
            'original_image': img,
            'image_filename': os.path.basename(img_path)
        }

# --- 2.3. 데이터 로더 Collate 함수 ---
def detr_collate_fn_train(batch):
    pixel_values = [item['pixel_values'] for item in batch]
    labels = [item['labels'] for item in batch]
    return {
        'pixel_values': torch.stack(pixel_values),
        'labels': labels
    }

def detr_collate_fn_test(batch):
    pixel_values = [item['pixel_values'] for item in batch]
    original_images = [item['original_image'] for item in batch]
    image_filenames = [item['image_filename'] for item in batch]
    return {
        'pixel_values': torch.stack(pixel_values),
        'original_images': original_images,
        'image_filenames': image_filenames
    }

# --- 2.4. 데이터셋 및 데이터 로더 인스턴스 생성 ---
train_dataset = CocoDetection(
    img_folder=config.train_image_dir,
    ann_file=config.train_annotation_file,
    image_processor=image_processor
)

test_dataset = TestImageDataset(
    img_folder=config.test_image_dir,
    image_processor=image_processor,
    category_map=train_dataset.label_to_name
)

train_dataloader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True, collate_fn=detr_collate_fn_train)
test_dataloader = DataLoader(test_dataset, batch_size=config.batch_size, shuffle=False, collate_fn=detr_collate_fn_test)

print(f"Train dataset size: {len(train_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")
print(f"Number of categories: {len(train_dataset.categories)}")
print(f"Categories: {train_dataset.label_to_name}")

loading annotations into memory...
Done (t=0.08s)
creating index...
index created!
Train dataset size: 1489
Test dataset size: 843
Number of categories: 73
Categories: {0: '보령부스파정 5mg', 1: '동아가바펜틴정 800mg', 2: '낙소졸정 500/20mg', 3: '신바로정', 4: '가바토파정 100mg', 5: '란스톤엘에프디티정 30mg', 6: '펠루비정(펠루비프로펜)', 7: '울트라셋이알서방정', 8: '비모보정 500/20mg', 9: '레일라정', 10: '스토가정 10mg', 11: '라비에트정 20mg', 12: '놀텍정 10mg', 13: '에스원엠프정 20mg', 14: '케이캡정 50mg', 15: '뮤테란캡슐 100mg', 16: '알드린정', 17: '타이레놀정500mg', 18: '삐콤씨에프정 618.6mg/병', 19: '다보타민큐정 10mg/병', 20: '트루비타정 60mg/병', 21: '메가파워정 90mg/병', 22: '비타비백정 100mg/병', 23: '타이레놀이알서방정(아세트아미노펜)(수출용)', 24: '리렉스펜정 300mg/PTP', 25: '써스펜8시간이알서방정 650mg', 26: '맥시부펜이알정 300mg', 27: '삼남건조수산화알루미늄겔정', 28: '큐시드정 31.5mg/PTP', 29: '일양하이트린정 2mg', 30: '뉴로메드정(옥시라세탐)', 31: '리피토정 20mg', 32: '크레스토정 20mg', 33: '오마코연질캡슐(오메가-3-산에틸에스테르90)', 34: '플라빅스정 75mg', 35: '아토르바정 10mg', 36: '리피로우정 20mg', 37: '리바로정 4mg', 38: '아토젯정 10/40mg', 39: '로수젯정10/5밀리그램', 40: '로수바미브정 10/20mg', 41: '에빅사정(메만틴염산염)(비매품)', 42: '리리카캡

In [ ]:
# --- 3.1. DETR 모델 로드 및 최적화기 설정 ---
model = DetrForObjectDetection.from_pretrained(
    config.model_name,
    num_labels=len(train_dataset.categories),
    ignore_mismatched_sizes=True
)
model.to(config.device)

optimizer = torch.optim.AdamW(model.parameters(), lr=config.learning_rate)

print(f"DETR model loaded with {len(train_dataset.categories)} labels.")

# --- 3.2. 모델 학습 루프 ---
print(f"Starting training on {config.device} for {config.epochs} epochs...")

for epoch in range(config.epochs):
    model.train()
    total_loss = 0
    for batch_idx, batch in enumerate(tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{config.epochs}")):
        pixel_values = batch['pixel_values'].to(config.device)
        labels = [{k: v.to(config.device) for k, v in t.items()} for t in batch['labels']]

        outputs = model(pixel_values=pixel_values, labels=labels)
        loss = outputs.loss

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch+1}, Average Loss: {avg_loss:.4f}")

    model_save_path = os.path.join(config.output_dir, f"detr_model_epoch_{epoch+1}.pth")
    torch.save(model.state_dict(), model_save_path)
    print(f"Model saved to {model_save_path}")

print("Training finished.")

# --- 3.3. 테스트 데이터셋 예측 및 시각화 ---
model.eval()
print("Starting prediction and visualization on test dataset (no ground truth)...")

with torch.no_grad():
    num_visualized = 0
    for batch_idx, batch in enumerate(tqdm(test_dataloader, desc="Predicting on test data")):
        if num_visualized >= config.viz_num_images:
            break

        pixel_values = batch['pixel_values'].to(config.device)
        original_images = batch['original_images']
        image_filenames = batch['image_filenames']

        outputs = model(pixel_values=pixel_values)
        logits = outputs.logits  # [batch_size, num_queries, num_classes]
        pred_boxes = outputs.pred_boxes  # [batch_size, num_queries, 4]

        for i in range(pixel_values.shape[0]):
            if num_visualized >= config.viz_num_images:
                break

            original_img = original_images[i]
            img_width, img_height = original_img.size
            img_filename = image_filenames[i]

            probas_i = logits[i].softmax(-1)  # [num_queries, num_classes]
            bboxes_i = pred_boxes[i]  # [num_queries, 4]

            # 예측 필터링: 배경 제외 + 임계값 이상
            scores = probas_i.max(-1).values
            labels = probas_i.argmax(-1)
            keep = (scores > config.detection_threshold) & (labels != len(train_dataset.categories) - 1)

            if keep.sum() == 0:
                print(f"[{img_filename}] No predictions passed threshold.")
                continue

            filtered_boxes = bboxes_i[keep]
            filtered_probas = probas_i[keep]

            # 좌표 복원 (cxcywh → xywh)
            filtered_boxes = filtered_boxes * torch.tensor(
                [img_width, img_height, img_width, img_height],
                device=filtered_boxes.device
            )
            boxes_xywh = torch.stack([
                filtered_boxes[:, 0] - filtered_boxes[:, 2] / 2,  # x_min
                filtered_boxes[:, 1] - filtered_boxes[:, 3] / 2,  # y_min
                filtered_boxes[:, 2],  # width
                filtered_boxes[:, 3]   # height
            ], dim=1).cpu().numpy()

            pred_labels = filtered_probas.argmax(-1).cpu().numpy()

            fig, ax = plt.subplots(1, figsize=(10, 10))
            ax.imshow(original_img)
            ax.set_title(f"Predicted Objects for {img_filename}", fontsize=12)
            ax.axis('off')

            for box, label_idx in zip(boxes_xywh, pred_labels):
                x_min, y_min, width, height = box
                if width > 0 and height > 0:
                    rect = patches.Rectangle(
                        (x_min, y_min), width, height,
                        linewidth=2, edgecolor='r', facecolor='none'
                    )
                    ax.add_patch(rect)
                    class_name = train_dataset.label_to_name.get(train_dataset.label_to_id[label_idx], 'Unknown')
                    ax.text(x_min, y_min - 5, class_name, color='r', fontsize=10,
                            bbox=dict(facecolor='white', alpha=0.7))

            viz_save_path = os.path.join(config.output_dir, f"prediction_{os.path.splitext(img_filename)[0]}.png")
            plt.savefig(viz_save_path, bbox_inches='tight', pad_inches=0.1)
            plt.close(fig)

            num_visualized += 1

print("Prediction and visualization finished. Results saved to output directory.")

Some weights of the model checkpoint at facebook/detr-resnet-50 were not used when initializing DetrForObjectDetection: ['model.backbone.conv_encoder.model.layer1.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing DetrForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DetrForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DetrForObjectDetection were not initialized from the model checkpoin

DETR model loaded with 73 labels.
Starting training on cpu for 1 epochs...


Epoch 1/1:  49%|████▉     | 736/1489 [1:00:56<1:13:27,  5.85s/it]

torch.Size([74])
